"Forecasting with a financial LLM" notes prepared for [CAS ETH in ML in Finance and Insurance](https://finsuretech.ethz.ch/continuing-education/cas-ml-in-finance-and-insurance.html) by Eleni Verteouri.


The current notebook uses the [FinGPT Project](https://github.cAI4Finance-Foundation/FinGPT) and has also available a [user interface](https://huggingface.co/spaces/FinGPT/FinGPT-Forecaster).

In [ ]:
!pip install torch==2.0.1
!pip install transformers==4.32.0
!pip install peft==0.5.0
!pip install pandas
!pip install yfinance
!pip install finnhub-python
!pip install nvidia-ml-py3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
!pip install datasets
!pip install peft
!pip install utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13905 sha256=e391fd296877a0d4a1293a5d9c6f30bf445f1102b3ff0eb4636a6a6a6d4c30de
  Stored in directory: /root/.cache/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils


In [ ]:
import torch
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from utils import *

In [ ]:
!pip install --upgrade huggingface_hub
from huggingface_hub import login
login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.2/346.2 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-2-7b-chat-hf',
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16,
)
base_model.model_parellal = True

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
model = PeftModel.from_pretrained(base_model, 'FinGPT/fingpt-forecaster_dow30_llama2-7b_lora')
model = model.eval()

adapter_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/40.1M [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')
tokenizer.padding_side = "right"
tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the path you want to check
import os
path_to_check = "/content/drive/My Drive/FinGPT-forecaster-AXP"

# Check if the specified path exists
if os.path.exists(path_to_check):
    print("Path exists.")
else:
    print("Path does not exist.")


Path exists.


In [ ]:
test_dataset = load_from_disk('/content/drive/MyDrive/FinGPT-forecaster-AXP')['test']

In [ ]:
def test_demo(model, tokenizer, prompt):

    inputs = tokenizer(
        prompt, return_tensors='pt',
        padding=False, max_length=4096
    )
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    res = model.generate(
        **inputs, max_length=4096, do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True
    )
    output = tokenizer.decode(res[0], skip_special_tokens=True)
    return output
    # return res

In [ ]:
import re

In [ ]:
answers, gts = [], []

for i in range(len(test_dataset)):
    prompt = test_dataset[i]['prompt']
    output = test_demo(model, tokenizer, prompt)
    answer = re.sub(r'.*\[/INST\]\s*', '', output, flags=re.DOTALL)
    gt = test_dataset[i]['answer']
    print('\n------- Prompt ------\n')
    print(prompt)
    print('\n------- LLaMA2 Finetuned ------\n')
    print(answer)
    print('\n------- GPT4 Groundtruth ------\n')
    print(gt)
    print('\n===============\n')
    answers.append(answer)
    gts.append(gt)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



------- Prompt ------

[INST]<<SYS>>
You are a seasoned stock market analyst. Your task is to list the positive developments and potential concerns for companies based on relevant news and basic financials from the past weeks, then provide an analysis and prediction for the companies' stock price movement for the upcoming week. Your answer format should be as follows:

[Positive Developments]:
1. ...

[Potential Concerns]:
1. ...

[Prediction & Analysis]
Prediction: ...
Analysis: ...

<</SYS>>

[Company Introduction]:

American Express Co is a leading entity in the Financial Services sector. Incorporated and publicly traded since 1977-05-18, the company has established its reputation as one of the key players in the market. As of today, American Express Co has a market capitalization of 133484.32 in USD, with 728.75 shares outstanding.

American Express Co operates primarily in the US, trading under the ticker AXP on the NEW YORK STOCK EXCHANGE, INC.. As a dominant force in the Financ

In [ ]:
gts[0]

"[Positive Developments]:\n1. Berkshire Hathaway and Ken Fisher have shown interest in American Express, indicating strong belief in the company's prospects.\n2. The rebound in the company’s share prices over the period of one week indicates investor sentiment could be turning positive.\n\n[Potential Concerns]:\n1. American Express was a major contributor to the Dow's fall, signaling a weak financial performance which could impact future stock price negatively.\n2. The presence of competitor Chase, opening its first U.S. airport lounge, suggests increasing competition for American Express.\n\n[Prediction & Analysis]:\nPrediction: Up by 2-3%\nAnalysis: American Express has been through a rough patch in the past week, contributing to the fall of the Dow. However, the interest expressed by Berkshire Hathaway and Ken Fisher in the company demonstrates their confidence in its future prospects. This could potentially have a significant effect on its stock price as it is seen in a positive li

In [ ]:
answers[0]

"[Positive Developments]:\n1. American Express's stock price increased from 146.35 to 151.31 between May 14th and 21st, suggesting a potential uptick in investor confidence.\n2. The company's presence in the financial services sector and its market capitalization of 133484.32 in USD indicate its strong financial position.\n\n[Potential Concerns]:\n1. The company's stock underperformed compared to its competitors on Tuesday, indicating potential weakness in the market.\n2. The company's stock price decreased from 151.02 to 146.35 between May 7th and 14th, indicating a potential downtrend.\n3. The company's stock is listed on the New York Stock Exchange, which is subject to market volatility and global economic trends.\n\n[Prediction & Analysis]:\nPrediction: Down by 2-3%\nAnalysis: While American Express has shown a positive trend in the past week, the overall market trends and potential concerns for the company suggest a possible downtrend in the upcoming week. The underperformance com

needs utils.py fingpt/FinGPT_Forecaster/utils.py

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install -r rouge/requirements.txt
!pip install rouge_score

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'rouge/requirements.txt'
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=973f4abf30f0e876d9f3461b9c6387707d198e70ed0b9c5fc26c5c90a14f1f5f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from sklearn.metrics import accuracy_score, mean_squared_error
from collections import defaultdict
from rouge_score import rouge_scorer


lora_module_dict = {
    'chatglm2': ['query_key_value'],
    'llama2': [
        'q_proj', 'k_proj', 'v_proj',
        'o_proj', 'gate_proj', 'up_proj', 'down_proj',
        # 'embed_tokens', 'lm_head',
    ],
}


def tokenize(args, tokenizer, feature):

    prompt_ids = tokenizer.encode(
        feature['prompt'].strip(), padding=False,
        max_length=args.max_length, truncation=True
    )

    target_ids = tokenizer.encode(
        feature['answer'].strip(), padding=False,
        max_length=args.max_length, truncation=True, add_special_tokens=False
    )

    input_ids = prompt_ids + target_ids
    exceed_max_length = len(input_ids) >= args.max_length

     # Add EOS Token
    if input_ids[-1] != tokenizer.eos_token_id and not exceed_max_length:
        input_ids.append(tokenizer.eos_token_id)

    label_ids = [tokenizer.pad_token_id] * len(prompt_ids) + input_ids[len(prompt_ids):]

    return {
        "input_ids": input_ids,
        "labels": label_ids,
        "exceed_max_length": exceed_max_length
    }


def parse_model_name(name, from_remote=False):

    if name == 'chatglm2':
        return 'THUDM/chatglm2-6b' if from_remote else 'base_models/chatglm2-6b'
    elif name == 'llama2':
        return 'meta-llama/Llama-2-7b-chat-hf' # if from_remote else 'base_models/Llama-2-7b-chat-hf'
    else:
        raise ValueError(f"Undefined base model {name}")


def load_dataset(names, from_remote=False):

    dataset_names = [d for d in names.split(',')]
    dataset_list = []

    for name in dataset_names:
        rep = 1
        if not os.path.exists(name):
            rep = int(name.split('*')[1]) if '*' in name else 1
            name = ('FinGPT/fingpt-forecaster-' if from_remote else 'data/fingpt-forecaster-') + name.split('*')[0]
        tmp_dataset = datasets.load_dataset(name) if from_remote else datasets.load_from_disk(name)

        if 'test' not in tmp_dataset:
            tmp_dataset = tmp_dataset.train_test_split(0.2, shuffle=True, seed=42)
        dataset_list.extend([tmp_dataset] * rep)

    return dataset_list


def parse_answer(answer):

    match_res = re.match(r"^\s*\[Positive Developments\]:\s*(.*)\s*\[Potential Concerns\]:\s*(.*)\s*\[Prediction (&|and) Analysis\]:\s*(.*)\s*$", answer, flags=re.DOTALL)
    if not match_res:
        return None

    pros, cons, pna = match_res.group(1), match_res.group(2), match_res.group(4)

    match_res = re.match(r'^Prediction:\s*(.*)\s*Analysis:\s*(.*)\s*$', pna, flags=re.DOTALL)
    if not match_res:
        return None

    pred, anal = match_res.group(1), match_res.group(2)

    if re.search(r'up|increase', pred.lower()):
        pred_bin = 1
    elif re.search(r'down|decrease|decline', pred.lower()):
        pred_bin = -1
    else:
        pred_bin = 0

    match_res = re.search(r'(\d)-(\d)%', pred)
    if not match_res:
        match_res = re.search(r'(?:more than )?(\d)+?%', pred)

    pred_margin = pred_bin * (int(match_res.group(1)) + 0.5) if match_res else 0.

    return {
        "positive developments": pros,
        "potential concerns": cons,
        "prediction": pred_margin,
        "prediction_binary": pred_bin,
        "analysis": anal
    }


def calc_rouge_score(references, answers):

    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    scores_per_pair = [scorer.score(ref, ans) for ref, ans in zip(references, answers)]

    rouge1 = sum(score['rouge1'].fmeasure for score in scores_per_pair) / len(scores_per_pair)
    rouge2 = sum(score['rouge2'].fmeasure for score in scores_per_pair) / len(scores_per_pair)
    rougeL = sum(score['rougeL'].fmeasure for score in scores_per_pair) / len(scores_per_pair)

    return {'rouge1': rouge1, 'rouge2': rouge2, 'rougeL': rougeL}


def calc_metrics(answers, gts):

    answers_dict = defaultdict(list)
    gts_dict = defaultdict(list)

    for answer, gt in zip(answers, gts):
        answer_dict = parse_answer(answer)
        gt_dict = parse_answer(gt)

        if answer_dict and gt_dict:
            for k in answer_dict.keys():
                answers_dict[k].append(answer_dict[k])
                gts_dict[k].append(gt_dict[k])

    if not answers_dict['prediction']:
        return {}

    bin_acc = accuracy_score(gts_dict['prediction_binary'], answers_dict['prediction_binary'])
    mse = mean_squared_error(gts_dict['prediction'], answers_dict['prediction'])

    pros_rouge_scores = calc_rouge_score(gts_dict['positive developments'], answers_dict['positive developments'])
    cons_rouge_scores = calc_rouge_score(gts_dict['potential concerns'], answers_dict['potential concerns'])
    anal_rouge_scores = calc_rouge_score(gts_dict['analysis'], answers_dict['analysis'])

    print(f"\nBinary Accuracy: {bin_acc:.2f}  |  Mean Square Error: {mse:.2f}")
    print(f"\nRouge Score of Positive Developments: {pros_rouge_scores}")
    print(f"\nRouge Score of Potential Concerns: {cons_rouge_scores}")
    print(f"\nRouge Score of Summary Analysis: {anal_rouge_scores}")

    return {
        "valid_count": len(answers_dict['prediction']),
        "bin_acc": bin_acc,
        "mse": mse,
        "pros_rouge_scores": pros_rouge_scores,
        "cons_rouge_scores": cons_rouge_scores,
        "anal_rouge_scores": anal_rouge_scores
    }


In [ ]:
calc_metrics(answers, gts)


Binary Accuracy: 0.50  |  Mean Square Error: 14.50

Rouge Score of Positive Developments: {'rouge1': 0.3924646781789639, 'rouge2': 0.11135218907400232, 'rougeL': 0.23390894819466249}

Rouge Score of Potential Concerns: {'rouge1': 0.28424938474159145, 'rouge2': 0.07478632478632478, 'rougeL': 0.18416735028712058}

Rouge Score of Summary Analysis: {'rouge1': 0.46901960784313723, 'rouge2': 0.18482955678118146, 'rougeL': 0.22999999999999998}


{'valid_count': 2,
 'bin_acc': 0.5,
 'mse': 14.5,
 'pros_rouge_scores': {'rouge1': 0.3924646781789639,
  'rouge2': 0.11135218907400232,
  'rougeL': 0.23390894819466249},
 'cons_rouge_scores': {'rouge1': 0.28424938474159145,
  'rouge2': 0.07478632478632478,
  'rougeL': 0.18416735028712058},
 'anal_rouge_scores': {'rouge1': 0.46901960784313723,
  'rouge2': 0.18482955678118146,
  'rougeL': 0.22999999999999998}}

results acceptable given we only choose one stock (AXP), compared to:
Binary Accuracy: 0.29  |  Mean Square Error: 4.64

Rouge Score of Positive Developments: {'rouge1': 0.4202282600542612, 'rouge2': 0.14931479501035597, 'rougeL': 0.2578559230218219}

Rouge Score of Potential Concerns: {'rouge1': 0.40652271907403026, 'rouge2': 0.13952370414379708, 'rougeL': 0.2513779865861211}

Rouge Score of Summary Analysis: {'rouge1': 0.4178582895911193, 'rouge2': 0.11082203617356211, 'rougeL': 0.2031596404349372}
{'valid_count': 14,
 'bin_acc': 0.2857142857142857,
 'mse': 4.642857142857143,
 'pros_rouge_scores': {'rouge1': 0.4202282600542612,
  'rouge2': 0.14931479501035597,
  'rougeL': 0.2578559230218219},
 'cons_rouge_scores': {'rouge1': 0.40652271907403026,
  'rouge2': 0.13952370414379708,
  'rougeL': 0.2513779865861211},
 'anal_rouge_scores': {'rouge1': 0.4178582895911193,
  'rouge2': 0.11082203617356211,
  'rougeL': 0.2031596404349372}}

In [ ]:
print(answers[0])


[Positive Developments]:
1. American Express's stock price increased from 146.35 to 151.31 between May 14th and 21st, suggesting a potential uptick in investor confidence.
2. The company's presence in the financial services sector and its market capitalization of 133484.32 in USD indicate its strong financial position.

[Potential Concerns]:
1. The company's stock underperformed compared to its competitors on Tuesday, indicating potential weakness in the market.
2. The company's stock price decreased from 151.02 to 146.35 between May 7th and 14th, indicating a potential downtrend.
3. The company's stock is listed on the New York Stock Exchange, which is subject to market volatility and global economic trends.

[Prediction & Analysis]:
Prediction: Down by 2-3%
Analysis: While American Express has shown a positive trend in the past week, the overall market trends and potential concerns for the company suggest a possible downtrend in the upcoming week. The underperformance compared to com